In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.odr import ODR, Model, Data, RealData
from scipy.optimize import curve_fit
import seaborn as sns
import uncertainties as uc
from uncertainties.umath import sqrt
from scipy.optimize import fsolve
from scipy.interpolate import interp1d

### Formatting

In [3]:
plt.style.use('Solarize_Light2')   #Options
                                    #Solarize_Light2
                                    #_classic_test_patch
                                    #_mpl-gallery
                                    #_mpl-gallery-nogrid
                                    #bmh
                                    #classic
                                    #dark_background
                                    #fast
                                    #fivethirtyeight
                                    #ggplot
                                    #grayscale
                                    #seaborn-v0_8
                                    #seaborn-v0_8-bright
                                    #seaborn-v0_8-colorblind
                                    #seaborn-v0_8-dark
                                    #seaborn-v0_8-dark-palette
                                    #seaborn-v0_8-darkgrid
                                    #seaborn-v0_8-deep
                                    #seaborn-v0_8-muted
                                    #seaborn-v0_8-notebook
                                    #seaborn-v0_8-paper
                                    #seaborn-v0_8-pastel
                                    #seaborn-v0_8-poster
                                    #seaborn-v0_8-talk
                                    #seaborn-v0_8-ticks
                                    #seaborn-v0_8-white
                                    #seaborn-v0_8-whitegrid
                                    #tableau-colorblind10

plt.rcParams.update({

    'figure.dpi': 225,                 # Resolution of figures
    'font.size': 10,                   # Default font size
    'font.family': 'sans-serif',       # Font family
    'font.sans-serif': ['DejaVu Sans'],
    
    'axes.titlesize': 10,              # Title font size
    'axes.labelsize': 8,              # Axis label font size
    'axes.linewidth': 0.8,             # Width of the axis lines
    'axes.grid': True,                 # Show grid
    # 'grid.color': '#dddddd',
    'grid.linestyle': '--',            # Grid line style (dashed)
    'grid.linewidth': 0.5,             # Grid line width
    
    'xtick.labelsize': 8,             # X-axis tick label size
    'ytick.labelsize': 8,             # Y-axis tick label size
    'xtick.direction': 'in',           # X-axis tick direction
    'ytick.direction': 'in',           # Y-axis tick direction

    'lines.linewidth': 0.8,            # Line width for plots
     'lines.markersize': 0.3,             # Marker size for scatter plots
     'lines.marker': 'o',               # Default marker shape

    'legend.frameon': True,            # Frame around legend
    'legend.framealpha': 0.8,          # Transparency of legend frame
    'legend.fontsize': 9,             # Font size in legend
    'legend.loc': 'best',              # Default legend location
})

In [4]:
# Errors functions
def rSquared(f_model, x, y, Params):
    y_pred = f_model(*Params, x)  
    ss_res = np.sum((y - y_pred) ** 2)  
    ss_tot = np.sum((y - np.mean(y)) ** 2)  
    return 1 - (ss_res / ss_tot)

def Residuals(y_measured,y_modelled):
    return y_measured - y_modelled

def MeanSquareError(y_measured,y_modelled):
    return ((1/len(y_measured))*(np.sum((y_measured-y_modelled)**2)))

def RootMeanSquareError(y_measured,y_modelled):
    return np.sqrt(MeanSquareError(y_measured,y_modelled))

def MeanAbsError(y_measured,y_modelled):
    return ((1/len(y_measured))*(np.sum(np.abs(y_measured-y_modelled))))

def ConfidenceInterval(y_measured,y_modelled,confInt=95):
    if(confInt == 95):
        Z = 1.96
    elif(confInt == 90):
        Z = 1.645
    elif(confInt == 99):
        Z = 2.576
    return Z*np.sqrt((1/len(y_measured))*(np.sum((y_measured-y_modelled)**2)))

def ArrayWithStaticUnc(Array,unc):
    a = []
    b = []
    for i in range(len(Array)):
            a.append(uc.ufloat(Array[i],unc))
            b.append(unc)
    return np.array(a), np.array(b)

def UnpackUncArray(Array):
     value = [Array[i].nominal_value for i in range(len(Array))]
     std = [Array[i].std_dev for i in range(len(Array))]
     return value,std 



# Non Linear Curve Fiting Function
def NonLinAnalysis(f_model, x, y, ParamsArr, weights):
    if ParamsArr is None:
        Params = None
        std = None 
        R2 = rSquared(f_model,x,y,Params)
    else:
        Params, cov = curve_fit(f_model, x, y, p0=ParamsArr, sigma=weights)
        std = np.sqrt(np.diag(cov))
        R2 = rSquared(f_model,x,y,Params)
    return Params, std, R2
 
def RR2(y,y_pred):
   ss_res = np.sum((y - y_pred) ** 2)
   ss_tot = np.sum((y - np.mean(y)) ** 2)
   R2 = 1 - (ss_res / ss_tot)
   return R2

### Basic Functions

In [5]:
L_T1 = uc.ufloat(0.348,0.00522)  # m # solenoid 1 length 
R_T1 = uc.ufloat(0.0131,(0.0131*0.0001965))  #m #solenoid 1 radius 
I_T1 = uc.ufloat(0.999,0.0035)  # A
n_T1 = 291 / L_T1 #m^-1 # coil density
def Magfield(z): #NOTE: Eq. 5 for Task 1
    MagFieldArray = []
    z = z
    for i in range(len(z)):
        a = ((mu0*I_T1*n_T1)/2)*((z[i]/(sqrt(R_T1**2+z[i]**2)))+((L_T1-z[i])/(sqrt(R_T1**2+(L_T1-z[i])**2))))
        MagFieldArray.append(a)
    return np.array(MagFieldArray)

R = uc.ufloat(0.068, 0.00102 ) #m
I = uc.ufloat(0.999, 0.003495 ) #A 
N = 320 
mu0 = 4*np.pi*10**(-7) #N/A^2

def Bz(z): #CHANGED: added brackets around everything before * 10**6 NOTE: Eq. 9 For Task 2
    BzArray = [] 
    for i in range(len(z)):
        b = ((mu0 * I * N * R**2) / (2 * (R**2 + z[i]**2)**(3/2))) * 10**6 #μT
        BzArray.append(b)
    return np.array(BzArray)

def Bz1(d, z): #NOTE: Eq. 10, For Task3
    BzArr = []
    for i in range(len(z)):
        f = 1 / (R**2 + (z[i] - (d/2))**2)**(3/2)
        e = 1 / (R**2 + (z[i] + (d/2))**2)**(3/2)
        c = ((mu0 * I * N * R**2)/2) * (f + e) * 10**6 #μT
        BzArr.append(c)
    return np.array(BzArr)

### Data Inputting

In [ ]:
Task1 = pd.read_csv('Task1.csv')
Task1["x'"] = Task1["x'"]*1e-3 
Task1["x'mitUnc"] = ArrayWithStaticUnc(Task1["x'"],0.5*1e-3)[0]
Task1["by"] = Task1["by"] + 23.6

Task2 = pd.read_csv('data/Task2.csv')
Task3 = pd.read_csv('data/Task3.csv')
halfR = pd.read_csv('data/halfR.csv')
Rvals = pd.read_csv('data/R.csv')
doubleR = pd.read_csv('data/doubleR.csv')

zi = Task2["x'"]*1e-3  #m
zimitUnc = ArrayWithStaticUnc(zi,0.5*10**(-3))[0]
bzmeas = Task2['by'] + 14.4 #μT

z1 = halfR["x'"] * 10**(-3) #m 
zimitUnc1 = ArrayWithStaticUnc(z1,0.5*10**(-3))[0]
bzmeas1 = halfR['by']+ 18.9  #μT

z2 = Rvals["x'"] * 10**(-3) #m
zimitUnc2 = ArrayWithStaticUnc(z2,0.5*10**(-3))[0] 
bzmeas2 = Rvals['by'] + 26.4 #μT

z3 = doubleR["x'"] * 10**(-3) #m
zimitUnc3 = ArrayWithStaticUnc(z3,0.5*10**(-3))[0]
bzmeas3 = doubleR['by'] + 15.9 #μT

## Task 1
1. Magnetic field distribution in a long solenoid
b. Plot both the measured and calculated magnetic induction in one graph.
c. Determine the magnetic induction at the end of the solenoid and compare experimental and
theoretical values.


In [ ]:
Task1

In [ ]:
MagField = np.array(Magfield(Task1["x'mitUnc"]))*1e6
MagField_nom, MagField_std = np.array(UnpackUncArray(Magfield(Task1["x'mitUnc"])))*1e6
R2_T1 = RR2(Task1["by"],MagField_nom)
plt.figure(figsize=(8, 5))
plt.ticklabel_format(axis='y', style='sci', scilimits=(1, 2))
# plt.ticklabel_format(axis='x', style='sci', scilimits=(0, -2))
plt.xticks([0.5,1.5,2.5,3.5],minor=True)
plt.plot(Task1["x'"],MagField_nom,color="red", label = "Theoretical")
plt.errorbar(Task1["x'"],Task1["by"],xerr=(np.ones(24)*0.5*1e-3),fmt='o',color="green")
plt.fill_between(Task1["x'"],MagField_nom-MagField_std, MagField_nom+MagField_std,
                    color="red",alpha=0.3,label="Theo. Uncertainty")
plt.plot(Task1["x'"],Task1["by"], color="green", label = "Measured")
plt.axvline(0, color='gray', linestyle='--', label='Start of solenoid')
plt.axvline(L_T1.nominal_value, color='gray', linestyle='--', label='End of solenoid')
plt.xlabel(r'Z (m)')
plt.ylabel(r'Magnetic Field $B(z)$ ($\mu$T)')
plt.title('Magnetic Field along Solenoid Axis')
Table_T1 = [[f'R²', f'{R2_T1:.4f}']]
plt.table(cellText= Table_T1,
                 loc='center',
                 cellLoc='center',
                 colWidths=[0.1,0.1],
                 alpha=0.7,
                 bbox=[0.65,0.1,0.2,0.05],zorder=100)
plt.legend(fontsize = 9)
plt.grid(True)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 6))
axs[1].ticklabel_format(axis='y', style='sci', scilimits=(1, 2))
# axs[1].ticklabel_format(axis='x', style='sci', scilimits=(0, -2))
axs[1].plot(Task1["x'"][20:24],MagField_nom[20:24],color="red", label = "Theoretical")
axs[1].errorbar(Task1["x'"][20:24],Task1["by"][20:24],xerr=(np.ones(4)*0.5*1e-3),fmt='o',color="green",capsize=2)
axs[1].fill_between(Task1["x'"][20:24],MagField_nom[20:24]-MagField_std[20:24],MagField_nom[20:24]+MagField_std[20:24],
            color="#e75480",alpha=0.3,label="Theo. Uncertainty")
axs[1].plot(Task1["x'"][20:24],Task1["by"][20:24], color="green", label = "Measured")
axs[1].scatter([.348,.348],[350,516],s=10,color="lightblue",edgecolors='k',linewidth=0.5,zorder=10,label = "Interpolated Values")
axs[1].annotate(r"$350\cdot{}\mu T$",[.3485,355],fontsize=6, zorder=10)
axs[1].annotate(r"$516\cdot{}\mu T$",[.3485,516.5],fontsize=6, zorder=10)
# axs[1].errorbar([347.5,347],[35,516],yerr=[33,5],fmt='o',color="lightblue")
axs[1].axvline(L_T1.nominal_value, color='gray', linestyle='--', label='End of solenoid')
axs[1].set_xlabel(r'Z (m)')
axs[1].set_ylabel(r'Magnetic Field $B(z)$ ($\mu$T)')
axs[1].set_title('Magnetic Field at End of Solenoid Axis')
axs[1].grid(True)
axs[1].legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(1, 2))
#plt.ticklabel_format(axis='x', style='sci', scilimits=(0, -2))


axs[0].ticklabel_format(axis='y', style='sci', scilimits=(1, 2))
# axs[0].ticklabel_format(axis='x', style='sci', scilimits=(-2, 2))
axs[0].plot(Task1["x'"][0:4],MagField_nom[0:4],color="red", label = "Theoretical")
axs[0].errorbar(Task1["x'"][0:4],Task1["by"][0:4],xerr=(np.ones(4)*0.5*1e-3),fmt='o',color="green",capsize=2)
axs[0].fill_between(Task1["x'"][0:4],MagField_nom[0:4]-MagField_std[0:4],MagField_nom[0:4]+MagField_std[0:4],
            color="#e75480",alpha=0.3,label="Theo. Uncertainty")
axs[0].plot(Task1["x'"][0:4],Task1["by"][0:4], color="green", label = "Measured")
axs[0].axvline(0, color='gray', linestyle='--', label='Start of solenoid')
axs[0].set_xlabel(r'Z (m)')
axs[0].set_ylabel(r'Magnetic Field $B(z)$ ($\mu$T)')
axs[0].set_title('Magnetic Field at Beginning of Solenoid Axis')
axs[0].grid(True)
axs[0].legend()

#axs[1].scatter([348],[0.000332],s=1,color="black",edgecolors='k',linewidth=0.5,zorder=10)
#axs[1].scatter([348],[0.000698],s=1,color="black",edgecolors='k',linewidth=0.5,zorder=10)
plt.show()

In [ ]:
# Finding gradient between two points on either side of 348, for both measured and calculated values to determined respective values at L=348mm
MeasuredGrad = (Task1["by"][21] - Task1["by"][20])/(Task1["x'mitUnc"][21] - Task1["x'mitUnc"][20])
TheoreticalGrad = (MagField[21] - MagField[20])/(Task1["x'mitUnc"][21] - Task1["x'mitUnc"][20])
print(MeasuredGrad)
print(TheoreticalGrad)
MeasuredInterpolatedEndValue = MeasuredGrad*(L_T1 - Task1["x'mitUnc"][20]) + (Task1["by"][20])
TheoreticalInterpolatedEndValue = TheoreticalGrad*(L_T1 - Task1["x'mitUnc"][20]) + MagField[20]
print()
print(MeasuredInterpolatedEndValue)
print(TheoreticalInterpolatedEndValue) 

In [ ]:
# a = [Task1["x'"][0],Task1["x'"][1],Task1["x'"][2],Task1["x'"][3],348,Task1["x'"][21],Task1["x'"][22],Task1["x'"][23]]
# b = [Task1["by"][0]*1e-6,Task1["by"][1]*1e-6,Task1["by"][2]*1e-6,Task1["by"][3]*1e-6,0.00035,Task1["by"][21]*1e-6,Task1["by"][22]*1e-6,Task1["by"][23]*1e-6]
# c = [MagField_nom[0],MagField_nom[1],MagField_nom[2],MagField_nom[3],0.000516,MagField_nom[21],MagField_nom[22],MagField_nom[23]]
a = [Task1["x'"][3],348]
b = [Task1["by"][3],350]
c = [MagField_nom[3],516]
cerr = [MagField_std[3],182]
plt.figure(figsize=(8,6),dpi=200)
plt.ticklabel_format(axis='y', style='sci', scilimits=(-2, 2))
plt.xticks(ticks=[0,348],labels=['Solenoid Start', 'Solenoid End'])
plt.xlim(-200,548)
plt.scatter(a,b,s=10,color="green",edgecolors='k',linewidth=0.5,zorder=10, label="Measured")
plt.scatter(a,c,s=10,color="#e75480",edgecolors='k',linewidth=0.5,zorder=10,label="Theoretical")
plt.errorbar(a,c,yerr=cerr,fmt="o",capsize=2,color="#e75480")
plt.legend()
plt.ylabel(r'Magnetic Field $B(z)$ ($\mu$T)')
plt.title("Solenoid End Points")
plt.show()

In [ ]:
measured_start = Task1['by'][3]
measured_end = 0.00035*1e6
calculated_start = MagField[3]
calculated_end = uc.ufloat(0.000516,0.000182)*1e6
print(f"Measured \n Start: {measured_start} \n End: {measured_end}")
print(f"Calculated \n Start: {calculated_start} \n End: {calculated_end}\n")


abs_error_start = abs(calculated_start - measured_start)
rel_error_start = (abs_error_start / measured_start) * 100

abs_error_end = abs(calculated_end - measured_end)
rel_error_end = (abs_error_end / measured_end) * 100


print(f"Start Absolute Error: {abs_error_start:.2f}")
print(f"Start Relative Error: {rel_error_start:.2f}%")
print(f"End Absolute Error: {abs_error_end:.2f}")
print(f"End Relative Error: {rel_error_end:.2f}%")

## Task 2

#### a. Measure the magnetic induction B(z) along the coil axis z of a flat solenoid.

In [ ]:
Task2

In [ ]:
plt.figure(figsize=(8,6),dpi=200)
plt.plot(zi, bzmeas)
plt.xlabel('z (m)')
# CHANGED: Added scilimit 
plt.ticklabel_format(axis='y', style='sci', scilimits=(-2, 2))
plt.ylabel('B (µT)')
plt.title('Measured Magnetic Induction along the z axis of the Flat Coil')
plt.show()

In [ ]:
plt.figure(figsize=(8,6),dpi=200)
bzcalc = np.array(Bz(zimitUnc))
bzcalc_nom, bzcalc_std = np.array(UnpackUncArray(bzcalc))
plt.plot(zi, bzcalc_nom, label='Calculated')
plt.xlabel('z (m)')
plt.ylabel('B (µT)')
plt.ticklabel_format(axis='y', style='sci', scilimits=(-2, 2))
plt.title('Calculated Magnetic Induction along the z axis of the Flat Coil')
plt.show()

#### b. Plot both the measured and calculated magnetic induction in one graph.

In [ ]:
R2 = RR2(bzmeas,bzcalc_nom)
plt.figure(figsize=(8, 5))
plt.ticklabel_format(axis='y', style='sci', scilimits=(1, 2))
# plt.ticklabel_format(axis='x', style='sci', scilimits=(0, -2))
plt.xticks([0.5,1.5,2.5,3.5],minor=True)
plt.plot(zi, bzcalc_nom, color="green", label = "Calculated")

plt.errorbar(zi , bzmeas, xerr=(np.ones(21)*0.5*1e-3),fmt='o',color="green")  
plt.fill_between(zi, bzcalc_nom - bzcalc_std, bzcalc_nom + bzcalc_std, color="#e75480",alpha=0.3,label="Uncertainty")
plt.plot(zi, bzmeas, color="blue", label = "Measured")

plt.table(cellText=[[f'R² = {R2:.4f}'], ],
                 loc='center left',
                 cellLoc='center',
                 colWidths=[0.2],
                 alpha=0.7)
plt.xlabel('z (m)')
plt.ylabel('B (µT)')
plt.title('Magnetic Induction along the z axis of One Flat Coil')
plt.legend()
plt.show()
print(bzcalc_std)

#### c. Determine at which distance from the coil center the maximum magnetic induction has decreased to half its value. Compare experimental and theoretical values.

In [ ]:
#theoretical 
bzmaxt = np.max(bzcalc_nom)
halfbzmaxt = 0.5 * bzmaxt
bzinterpt = interp1d(zi, bzcalc_nom, kind='cubic', fill_value="extrapolate")
def equation(zi):
    return bzinterpt(zi) - halfbzmaxt
z_g = zi[np.argmax(bzcalc_nom)] 
zhalft = fsolve(equation, R.nominal_value) 
print(f"Theoretical value of the distance from the center of the coil where the magnetic field is half of its maximum value is: ", zhalft[0])

#experimental
bzmaxe = np.max(bzmeas)
halfbzmaxe = 0.5 * bzmaxe
bz_interp = interp1d(zi, bzmeas, kind='cubic', fill_value="extrapolate")
def equation(zi):
    return bz_interp(zi) - halfbzmaxe
z_g = zi[np.argmax(bzmeas)] 
zhalfe= fsolve(equation, z_g)

print(f"Experimental value of the distance from the center of the coil where the magnetic field is half of its maximum value is: {zhalfe[0]:.8f} meters.")


### Task 3

#### a. Measure the axial magnetic induction B(z) of a pair of flat coils for three distances b between the coils: b = 2R, b = R, b = R/2 (R: coil radius).

In [ ]:
plt.plot(z1, bzmeas1, label='d = R/2', color='red')
plt.plot(z2, bzmeas2, label='d = R', color='green')
plt.plot(z3, bzmeas3, label='d = 2R', color='blue')
plt.xlabel('z (m)')
plt.ylabel('B (µT)')
#CHANGED: Added Scilimits
plt.ticklabel_format(axis='y', style='sci', scilimits=(-2, 2))
plt.title('Measured Magnetic Induction')
plt.legend()
plt.show()

#### b. Plot both the measured and calculated magnetic induction. 

In [ ]:
d1 = uc.ufloat(0.034, 0.00051) #m
bzcalc1 = np.array(Bz1(d1, zimitUnc1))
bzcalc_nom1, bzcalc_std1 = np.array(UnpackUncArray(bzcalc1))
R21 = RR2(bzmeas1,bzcalc_nom1)


plt.figure(figsize=(8, 5))
plt.ticklabel_format(axis='y', style='sci', scilimits=(1, 2))
# plt.ticklabel_format(axis='x', style='sci', scilimits=(0, -2))
plt.xticks([0.5,1.5,2.5,3.5],minor=True)

plt.plot(z1, bzcalc_nom1, label='Calculated', color='green')
plt.errorbar(z1 , bzmeas1, xerr=(np.ones(21)*0.5*1e-3),fmt='o',color="green")
plt.fill_between(z1, bzcalc_nom1 - bzcalc_std1, bzcalc_nom1 + bzcalc_std1, color="#e75480",alpha=0.3,label="Uncertainty")
plt.plot(z1, bzmeas1, color="blue", label = "Measured")

plt.title('Magnetic Induction for Two Coil at distance D = R/2')
plt.table(cellText=[[f'R² = {R21:.4f}']],
                 loc='center left',
                 cellLoc='center',
                 colWidths=[0.2],
                 alpha=0.7)
plt.xlabel('z (m)')
plt.ylabel('B (µT)')
plt.legend()
plt.show()


In [ ]:
d2 = uc.ufloat(0.068, 0.00102) #m
bzcalc2 = np.array(Bz1(d2, zimitUnc2))
bzcalc_nom2, bzcalc_std2 = np.array(UnpackUncArray(bzcalc2))
R22 = RR2(bzmeas2,bzcalc_nom2)

plt.figure(figsize=(8, 5))
plt.ticklabel_format(axis='y', style='sci', scilimits=(1, 2))
# plt.ticklabel_format(axis='x', style='sci', scilimits=(0, -2))
plt.xticks([0.5,1.5,2.5,3.5],minor=True)

plt.plot(z2, bzcalc_nom2, label='Calculated', color='green')
plt.errorbar(z2 , bzmeas2, xerr=(np.ones(21)*0.5*1e-3),fmt='o',color="green")
plt.fill_between(z2, bzcalc_nom2 - bzcalc_std2, bzcalc_nom2 + bzcalc_std2, color="#e75480",alpha=0.3,label="Uncertainty")
plt.plot(zi, bzmeas2, color="blue", label = "Measured")

plt.title('Magnetic Induction for Two Coil at distance D = R')
plt.table(cellText=[[f'R² = {R22:.4f}']],
                 loc='center',
                 cellLoc='center',
                 colWidths=[0.2],
                 alpha=0.7)
plt.xlabel('z (m)')
plt.ylabel('B (µT)')
plt.legend()
plt.show()



In [ ]:
d3 = uc.ufloat(0.136, 0.00241) #m
bzcalc3 = np.array(Bz1(d3, zimitUnc3))
bzcalc_nom3, bzcalc_std3 = np.array(UnpackUncArray(bzcalc3))
R23 = RR2(bzmeas3,bzcalc_nom3)

plt.figure(figsize=(8, 5))
plt.ticklabel_format(axis='y', style='sci', scilimits=(1, 2))
# plt.ticklabel_format(axis='x', style='sci', scilimits=(0, -2))
plt.xticks([0.5,1.5,2.5,3.5],minor=True)

plt.plot(z3, bzcalc_nom3, label='Calculated', color='green')
plt.errorbar(z3 , bzmeas3, xerr=(np.ones(25)*0.5*1e-3),fmt='o',color="green")
plt.fill_between(z3, bzcalc_nom3-bzcalc_std3, bzcalc_nom3+bzcalc_std3,
                    color="#e75480",alpha=0.3,label="Uncertainty")
plt.plot(z3, bzmeas3, color="blue", label = "Measured")

plt.title('Magnetic Induction for Two Coil at distance D = 2R')
plt.table(cellText=[[f'R² = {R23:.4f}']],
                 loc='lower right',
                 cellLoc='center',
                 colWidths=[0.2],
                 alpha=0.7)
plt.xlabel('z (m)')
plt.ylabel('B (µT)')
plt.legend()
plt.show()


#### c. For the Helmholtz configuration, determine theoretically and experimentally the region with a deviation of the magnetic induction less than 5% from its central value.

In [ ]:
#theoretical
b01 = (mu0 * N * I) / ((17 / 16)**(3/2) * R) * 10**6
bzinterest = interp1d(z1, bzcalc_nom1, kind='cubic', fill_value="extrapolate")
def deviation(z1):
    return abs(bzinterest(z1) - b01.nominal_value) - 0.05 * b01.nominal_value
z_positive = fsolve(deviation, [0.05])

print(f"The theoretical region with less than 5% deviation is", z_positive)

#experimental
Be0 = bzmeas1[np.argmin(np.abs(z1))]

tolerance = 0.05 * Be0
deviation = np.abs(bzmeas1 - Be0)
valid_indices = np.where(deviation <= tolerance)[0]

z_min = z1[valid_indices[0]]
z_max = z1[valid_indices[-1]]
print(f"The experimental region with less than 5% deviation is from {z_max:.8f} to {z_min:.8f} meters.")

In [ ]:
print(valid_indices)
# print(z1)
print(np.abs(bzmeas1 - Be0))
print(Be0 * 0.05) 

In [ ]:
#theoretical
b02 = (mu0 * N * I) / ((5 / 4)**(3/2) * R)  * 10**6
bzinterest2 = interp1d(z2, bzcalc_nom2, kind='cubic', fill_value="extrapolate")
def deviation(z1):
    return abs(bzinterest2(z1) - b02.nominal_value) - 0.05 * b01.nominal_value
z_positive2 = fsolve(deviation, [0.05])
print(f"The theoretical region with less than 5% deviation is",  z_positive2)

#experimental
Be01 = bzmeas2[np.argmin(np.abs(z2))]

tol1 = 0.05 * Be01
dev1 = np.abs(bzmeas2 - Be01)
valid1 = np.where(dev1 <= tol1)[0]

z_min1 = z2[valid1[0]]
z_max1 = z2[valid1[-1]]
print(f"The experimental region with less than 5% deviation is from {z_max1:.8f} to {z_min1:.8f} meters.")

In [ ]:
#theoretical
b03 = (mu0 * N * I) / ((2)**(3/2) * R) * 10**6
bzinterest3 = interp1d(z3, bzcalc_nom3, kind='cubic', fill_value="extrapolate")
def deviation(z1):
    return abs(bzinterest3(z1) - b03.nominal_value) - 0.05 * b01.nominal_value
z_positive3 = fsolve(deviation, [0.05])
print(f"The theoretical region with less than 5% deviation is",  z_positive3)

#experimental
Be02 = bzmeas3[np.argmin(np.abs(z3))]

tol2 = 0.05 * Be02
dev2 = np.abs(bzmeas3 - Be02)
valid2 = np.where(dev2 <= tol2)[0]
z_min2 = z3[valid2[0]]
z_max2 = z3[valid2[-1]]
print(f"The experimental region with less than 5% deviation is from -0.01 to 0.02 meters.")

In [ ]:
#Important Analysis
print(bzmeas3[np.argmin(np.abs(z3))])
print(bzmeas3[12]*0.05)

print(dev2)
print(bzmeas3)